# Extracting data for secondary task by filtering the genus based on New England (recomended dataset by clients, most representative of common genera) GBIF dataset


In [ ]:
pip install python-dwca-reader

In [1]:
import requests
import shutil
import os
import time
import math
import pandas
import json
from tqdm import tqdm
import glob
import random
from PIL import Image
import imghdr
from sklearn.model_selection import train_test_split
# Multiprocessing stuff
import multiprocessing as mp
NUM_CORES = min(mp.cpu_count(), 50)

# Some CV/DL module
import numpy as np
np.__version__
import matplotlib.pyplot as plt
import cv2


from dwca.read import DwCAReader
from dwca.darwincore.utils import qualname as qn

In [2]:
# Some hyper-parameters
EPOCHS = 25
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = './data'
width=256
height=256
depth=3


timestr = time.strftime("%Y%m%d-%H%M%S")
Diverse_genera = "/usr4/ds549/rithik/Desktop/new_england.csv"


DATASET_CSV = "//projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data/data.csv"
DATASET_PATH = org_img_dir = "/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data/scraped-data/20220616-061434/"	# "/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data/scraped-data/drago_testdata/"
craft_res_dir = org_img_dir.replace('/scraped-data/', '/CRAFT-results/')
seg_img_dir = org_img_dir.replace('/scraped-data/', '/seg-results/')

PERCENT_TO_SCRAPE = 0.0007
#PERCENT_TO_SCRAPE = 0.01252703
NUMBER_TO_SKIP = 0
OUTPUT_PATH = "/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/TrOCR evaluation data/" + timestr + "/"
TEXT_PATH = "/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/TrOCR evaluation data/"
OUTPUT_PATH_IMAGES = "/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/secondary_task/images/" + timestr + "/"
DATASET_URL = "https://occurrence-download.gbif.org/occurrence/download/request/0196625-210914110416597.zip"
DATASET_TYPE = ""


Note: There are 7,982,741 images avaialble in GBIF database, but we only scrapped 1198 images for evaluation purpose. If you want to see the amount of data that we have, use the following code, or you can checkout the 'scraping' folder to scrape more data from the website.

```python
# If you want to read the data from that data.csv file
import pandas as pd
df = pd.read_csv(DATASET_PATH + DATASET_CSV, sep="\t")
DATASET_TYPE = "csv"
colnames = []
for col in df.columns:
    colnames.append(col)
print(f"colnames: {colnames}")
print(f"df.shape: {df.shape}")
```



## Converting the CSV data into a pandas data frame

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
df = pd.read_csv(DATASET_CSV, sep="\t")
DATASET_TYPE = "csv"
colnames = []
for col in df.columns:
    colnames.append(col)
print(f"colnames: {colnames}")
print(f"df.shape: {df.shape}")

/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (2,14,16,17,19,21,24,25,26,32,33,34,36,37,38,39,40,41,43,45,46) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


colnames: ['gbifID', 'datasetKey', 'occurrenceID', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species', 'infraspecificEpithet', 'taxonRank', 'scientificName', 'verbatimScientificName', 'verbatimScientificNameAuthorship', 'countryCode', 'locality', 'stateProvince', 'occurrenceStatus', 'individualCount', 'publishingOrgKey', 'decimalLatitude', 'decimalLongitude', 'coordinateUncertaintyInMeters', 'coordinatePrecision', 'elevation', 'elevationAccuracy', 'depth', 'depthAccuracy', 'eventDate', 'day', 'month', 'year', 'taxonKey', 'speciesKey', 'basisOfRecord', 'institutionCode', 'collectionCode', 'catalogNumber', 'recordNumber', 'identifiedBy', 'dateIdentified', 'license', 'rightsHolder', 'recordedBy', 'typeStatus', 'establishmentMeans', 'lastInterpreted', 'mediaType', 'issue']
df.shape: (7982741, 50)


## Keeping only the rows of the GBIF data which match the genera in the new england new_england.csv

In [4]:
import pandas as pd

# Read in the dataframe df2 and extract genera list
df2 = pd.read_csv(Diverse_genera, sep="\t")
genera = df2['Genus'].tolist()
#print(genera)

# Make genera list lowercase
genera_lower = [genus.lower() for genus in genera]
#print(genera_lower)

# Make genus column of df lowercase
df['genus_lower'] = df['genus'].str.lower()
#print(df['genus_lower'])

# Filter df based on lowercase genus column
df = df[df['genus_lower'].isin(genera_lower)]

# Remove the lowercase genus column
df = df.drop('genus_lower', axis=1)

## Taking a look at the data csv

In [33]:
df.head()

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
0,2417026544,af5f680a-e0cc-46c8-b623-ceeaab70aa9e,MHA0027832,Plantae,Tracheophyta,Liliopsida,Poales,Cyperaceae,Eriophorum,Eriophorum angustifolium,...,Syreyschikov D.,NaN,CC_BY_4_0,GBS RAN - Glavny Botanichesky Sad Rossijskoj A...,Syreyschikov D. | Smirnov P.,NaN,NaN,2022-03-25T12:48:46.119Z,StillImage,DIFFERENT_OWNER_INSTITUTION
1,2417026527,af5f680a-e0cc-46c8-b623-ceeaab70aa9e,MHA0027833,Plantae,Tracheophyta,Liliopsida,Poales,Cyperaceae,Eriophorum,Eriophorum angustifolium,...,Syreyschikov D.,NaN,CC_BY_4_0,GBS RAN - Glavny Botanichesky Sad Rossijskoj A...,Syreyschikov D. | Smirnov P.,NaN,NaN,2022-03-25T12:48:46.120Z,StillImage,DIFFERENT_OWNER_INSTITUTION
2,2458329530,af5f680a-e0cc-46c8-b623-ceeaab70aa9e,MHA0003877,Plantae,Tracheophyta,Liliopsida,Alismatales,Hydrocharitaceae,Elodea,Elodea canadensis,...,Identified by the collector,NaN,CC_BY_4_0,GBS RAN - Glavny Botanichesky Sad Rossijskoj A...,S.S. Ganeshin,NaN,NaN,2022-03-25T12:48:46.136Z,StillImage,DIFFERENT_OWNER_INSTITUTION
3,2417025838,af5f680a-e0cc-46c8-b623-ceeaab70aa9e,MHA0027817,Plantae,Tracheophyta,Liliopsida,Poales,Cyperaceae,Eriophorum,Eriophorum angustifolium,...,Syreyschikov D.,NaN,CC_BY_4_0,GBS RAN - Glavny Botanichesky Sad Rossijskoj A...,Syreyschikov D. | Smirnov P.,NaN,NaN,2022-03-25T12:48:46.142Z,StillImage,DIFFERENT_OWNER_INSTITUTION
4,2458328836,af5f680a-e0cc-46c8-b623-ceeaab70aa9e,MHA0003842,Plantae,Tracheophyta,Liliopsida,Alismatales,Butomaceae,Butomus,Butomus umbellatus,...,Chernyakovskaya-Reyneke,NaN,CC_BY_4_0,GBS RAN - Glavny Botanichesky Sad Rossijskoj A...,Chernyakovskaya-Reyneke,NaN,NaN,2022-03-25T12:48:46.168Z,StillImage,DIFFERENT_OWNER_INSTITUTION


In [34]:
print(len(df))

4787138


## Looking at the most represented genus' in our data (and who collected the species)

In [35]:
x = df['genus'].value_counts()
print(f"We have {len(x)} genus in total!")
x[:-250]

We have 1051 genus in total!


Carex         228853
Salix          78596
Euphorbia      60624
Juncus         57851
Viola          49885
               ...  
Lycopus         4850
Ambrosia        4817
Neottia         4766
Cryptantha      4759
Agalinis        4705
Name: genus, Length: 250, dtype: int64

## Keeping only the first 400 occurences of each of the 1051 genera, to create a dataset of 420K images (due to download failures this number will reduce)

In [5]:
# create a list of unique genus names
clip = df['genus'].unique().tolist()

# create an empty list to store the clipped DataFrames
clipped_dfs = []

# iterate over each genus in the clip list
for genus in clip:
    # select rows from new_df where genus is equal to the current genus
    df_genus = df[df['genus'] == genus]
    # clip the number of rows to a maximum of 400
    df_genus_clipped = df_genus[:400]
    # append the clipped DataFrame to the clipped_dfs list
    clipped_dfs.append(df_genus_clipped)

# concatenate the clipped DataFrames into a single DataFrame
df_clip = pd.concat(clipped_dfs)

# display the new DataFrame
df_clip.head()

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
0,2417026544,af5f680a-e0cc-46c8-b623-ceeaab70aa9e,MHA0027832,Plantae,Tracheophyta,Liliopsida,Poales,Cyperaceae,Eriophorum,Eriophorum angustifolium,...,Syreyschikov D.,NaN,CC_BY_4_0,GBS RAN - Glavny Botanichesky Sad Rossijskoj A...,Syreyschikov D. | Smirnov P.,NaN,NaN,2022-03-25T12:48:46.119Z,StillImage,DIFFERENT_OWNER_INSTITUTION
1,2417026527,af5f680a-e0cc-46c8-b623-ceeaab70aa9e,MHA0027833,Plantae,Tracheophyta,Liliopsida,Poales,Cyperaceae,Eriophorum,Eriophorum angustifolium,...,Syreyschikov D.,NaN,CC_BY_4_0,GBS RAN - Glavny Botanichesky Sad Rossijskoj A...,Syreyschikov D. | Smirnov P.,NaN,NaN,2022-03-25T12:48:46.120Z,StillImage,DIFFERENT_OWNER_INSTITUTION
3,2417025838,af5f680a-e0cc-46c8-b623-ceeaab70aa9e,MHA0027817,Plantae,Tracheophyta,Liliopsida,Poales,Cyperaceae,Eriophorum,Eriophorum angustifolium,...,Syreyschikov D.,NaN,CC_BY_4_0,GBS RAN - Glavny Botanichesky Sad Rossijskoj A...,Syreyschikov D. | Smirnov P.,NaN,NaN,2022-03-25T12:48:46.142Z,StillImage,DIFFERENT_OWNER_INSTITUTION
8,2417027199,af5f680a-e0cc-46c8-b623-ceeaab70aa9e,MHA0027874,Plantae,Tracheophyta,Liliopsida,Poales,Cyperaceae,Eriophorum,Eriophorum latifolium,...,Syreyschikov D.,NaN,CC_BY_4_0,GBS RAN - Glavny Botanichesky Sad Rossijskoj A...,Smirnov P.,NaN,NaN,2022-03-25T12:48:46.193Z,StillImage,DIFFERENT_OWNER_INSTITUTION
9,2417024881,af5f680a-e0cc-46c8-b623-ceeaab70aa9e,MHA0027863,Plantae,Tracheophyta,Liliopsida,Poales,Cyperaceae,Eriophorum,Eriophorum gracile,...,Syreyschikov D.,NaN,CC_BY_4_0,GBS RAN - Glavny Botanichesky Sad Rossijskoj A...,Smirnov P.,NaN,NaN,2022-03-25T12:48:46.196Z,StillImage,DIFFERENT_OWNER_INSTITUTION


In [6]:
df = df_clip.copy()
del df_clip
x = df['genus'].value_counts()
print(f"We have {len(x)} genus in total!")
print("num of rows we need to extract",len(df))
tragus_count = x['Tragus']
print(tragus_count)

We have 1051 genus in total!
num of rows we need to extract 395043
400


## Extracting the ID values from the dataframe and storing it into a dictionary 'data'

In [7]:
data = {}
print(str(len(df)) + " IDs will be scraped.")

for i in range(1, df.shape[0]):
    try:
        if DATASET_TYPE == "dwca":
            id = df.at[i, "id"]
        elif DATASET_TYPE == "csv":
            id = df.at[i, "gbifID"]
        data[i] = {"id": str(id)}
    except KeyError:
        pass
print("Successfully scraped " + str(len(data)) + " IDs.")


395043 IDs will be scraped.
Successfully scraped 140562 IDs.


## Scrape the data using GBIF API and extract the image URL and genus information

In [9]:
#code to manually delete the data for corrupted or missing values
from itertools import islice
#del data[next(islice(data, 990, None))]


print("Data will be fetched for", len(data), "occurrences.")


def scrape_occurrence(key):
    try:
        rq = requests.get("https://api.gbif.org/v1/occurrence/" + str(data[key]["id"]))
        return_dict = {}
        return_dict[key] = {}

        if 'media' in json.loads(rq.content):
            if not json.loads(rq.content)["media"]: #checking for empty list
            #print(json.loads(rq.content)["media"][0].keys())
                return None
            else:
                if 'identifier' in json.loads(rq.content)["media"][0]:
                    return_dict[key]["img_url"] = json.loads(rq.content)["media"][0]["identifier"]
                else:
                    return None
        else:
            return None
            #return_dict[key]["img_url"] = "NAN"

        if 'format' in json.loads(rq.content)["media"][0]:
            return_dict[key]["img_type"] = json.loads(rq.content)["media"][0]["format"]
        else:
            return None   

        if 'country' in json.loads(rq.content):
            return_dict[key]["country"] = json.loads(rq.content)["country"]
        else:
            return None

        if 'genus' in json.loads(rq.content):
            return_dict[key]["genus"] = json.loads(rq.content)["genus"]
        else:
            return None
            #return_dict[key]["genus"] = "NAN"

        if 'species' in json.loads(rq.content):
            return_dict[key]["species"] = json.loads(rq.content)["species"]
        else:
            return None

        if 'recordedBy' in json.loads(rq.content):
            return_dict[key]["Collector"] = json.loads(rq.content)["recordedBy"]
        else:
            return None

        #print(return_dict)
        return return_dict
    
    except Exception as e:
        print("Error occurred for key:", key)
        print(str(e))
        return None
    
    #print(return_dict)
    return return_dict

scrape = {}
print("Starting multiprocessing...")
pool = mp.Pool(NUM_CORES)
print("Fetching data...")
#index = 1
for item in tqdm(pool.imap(scrape_occurrence, data), total=len(data)):
    if item==None:
        pass
    else:
        scrape.update(item)
    
pool.close()
pool.join()

print("\nSuccessfully fetched data for", len(scrape), "occurrences.")

# for values in scrape.values(): 
#    print(values)


Data will be fetched for 140562 occurrences.
Starting multiprocessing...


  0%|          | 0/140562 [00:00<?, ?it/s]

Fetching data...


  3%|▎         | 4880/140562 [01:32<28:48, 78.48it/s]  

Error occurred for key: 5544
Expecting value: line 1 column 1 (char 0)


...

  7%|▋         | 10128/140562 [03:10<35:29, 61.25it/s] 

Error occurred for key: 13130
Expecting value: line 1 column 1 (char 0)


 69%|██████▉   | 96979/140562 [29:24<17:23, 41.78it/s]   

Error occurred for key: 272769
Expecting value: line 1 column 1 (char 0)


 98%|█████████▊| 138340/140562 [41:48<00:50, 43.92it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Downloading the images and storing them along with Country, Collector and Taxon information

In [10]:
from time import sleep
import requests
import shutil
import multiprocessing as mp
import os
from functools import partial

def download(key):
    try:
        img = requests.get(scrape[key]["img_url"], stream=True, timeout=10)
        with open(
            OUTPUT_PATH_IMAGES + str(key) + "." + scrape[1]["img_type"].split("/", 1)[1], "wb"
        ) as f:
            shutil.copyfileobj(img.raw, f)
        return True, key
    except:
        return False, key

data = scrape.copy()

if not os.path.exists(OUTPUT_PATH_IMAGES):
    os.makedirs(OUTPUT_PATH_IMAGES)

print("Starting multiprocessing...")
pool = mp.Pool(NUM_CORES)
print("Downloading images...")
    #func = partial(download, data=data)
    #errorCount = 0
errorKeys = []
for success, key in tqdm(pool.map(download, scrape), total=len(scrape)):
    if success == False:
        errorKeys.append(key)
pool.close()
pool.join()
for key in errorKeys:
    del scrape[key]
    ## Loop to reindex downloaded images sequentially
    # for count, filename in enumerate(os.listdir(OUTPUT_PATH)):
    #     new = str(count) + "." + filename.split(".")[1]  # new file name
    #     src = os.path.join(OUTPUT_PATH, filename)  # file source
    #     dst = os.path.join(OUTPUT_PATH, new)  # file destination
    #     # rename all the file
    #     os.rename(src, dst)
print("\nSuccessfully downloaded", len(scrape), "images")




Starting multiprocessing...


100%|██████████| 129174/129174 [00:00<00:00, 2721329.96it/s]



Successfully downloaded 119138 images


## There are files that do not download due to url issues, multiprocessing issues. These corrupted images need to be deleted.

In [16]:
image_folder = "/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/secondary_task/images/20230402-114028/"

# Get the list of image files in the folder
image_files = os.listdir(image_folder)

# Iterate over the dictionary keys and check if the corresponding image file exists
missing_keys = []
for key in data.keys():
    image_filename = str(key) + '.jpg'
    if image_filename not in image_files:
        missing_keys.append(key)
print(f"based on image names we get {len(missing_keys)} keys that had a download failure.")
print("error keys: ",len(errorKeys))
delete_keys = [key for key in errorKeys if key not in missing_keys]
print("Delete these images from the created images folder if any due to multiprocess issues: ",delete_keys)




based on image names we get 9939 keys that had a download failure.
error keys:  10036
Delete these images from the created images folder if any due to multiprocess issues:  [1, 113, 259, 361, 467, 576, 682, 795, 1003, 1173, 1220, 1358, 1451, 1540, 1664, 1776, 1786, 1884, 2002, 2115, 2229, 2319, 2415, 2553, 2652, 2755, 2888, 3001, 3114, 3225, 3334, 3437, 3544, 3711, 3851, 3957, 4100, 4204, 4328, 4428, 4628, 4731, 4821, 5005, 5150, 5283, 5425, 5532, 5679, 5837, 5875, 5983, 6147, 6259, 6361, 6516, 6636, 6748, 6855, 6974, 7094, 7270, 7410, 7584, 7814, 8035, 8174, 8315, 8458, 8605, 8757, 8922, 9058, 9175, 9347, 9504, 9678, 9827, 10015, 10222, 10417, 10766, 10907, 11212, 11407, 11612, 11743, 11902, 12053, 12547, 12670, 12799, 12996, 13152, 13292, 13413, 13530]


In [28]:

image_folder = "/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/secondary_task/images/20230402-114028/"
#code to delete the corrupted images
for file_name in os.listdir(image_folder):
    file_key = file_name.split(".")[0]
    if int(file_key) in delete_keys:
        file_path = os.path.join(image_folder, file_name)
        print(file_path, "\n")
        os.remove(file_path)


/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/secondary_task/images/20230402-114028/2888.jpg 

/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/secondary_task/images/20230402-114028/2652.jpg 

/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/secondary_task/images/20230402-114028/4428.jpg 

/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/secondary_task/images/20230402-114028/795.jpg 

/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/secondary_task/images/20230402-114028/1884.jpg 

/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/secondary_task/images/20230402-114028/12799.jpg 

/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/secondary_task/images/20230402-114028/1358.jpg 

/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/secondary_task/images/20230402-114028/1664.jpg 

/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/secondary_task

## Making a CSV file to keep a log out of the scraped data- data.csv

In [17]:
import csv
with open("/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/secondary_task/data.csv", "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Key", "Value"])
    for key, value in scrape.items():
        writer.writerow([key, value])

In [13]:
TEXT_PATH = "/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/secondary_task/"
with open(TEXT_PATH + "genus.txt", "w") as f:
    for idx in scrape:
        f.write(scrape[idx]["genus"] + "\n")
print("\nSuccessfully wrote genus to file.")


Successfully wrote genus to file.
